In [2]:
import wrf_management.utilities as ut
import importlib
importlib.reload(ut);
import wrf_management.project_global_constants as gc
importlib.reload(gc)
import wrf_management.geogrid as geo
import os
import sqlite3 as sq
import pandas as pd
import wrf_management.base_namelists.base_namelists as bn
importlib.reload(bn);
import f90nml

In [3]:
con = sq.connect(gc.PATH_DB)

In [8]:
importlib.reload(ut)
program = 'ungrib'
unique_id = ut.get_unique_id(program)
print(unique_id)
unique_id = '2019-02-21T17-29-01_753696_ungrib'

2019-02-21T18-53-20_087667_ungrib


In [9]:
importlib.reload(ut)
new_path = ut.get_new_unique_path_for_program(unique_id=unique_id)
print(new_path)
new_path = '/tmp/wrf_management/data_folder/runs/2018_02_19/2019-02-21T17-29-01_753696_ungrib'

/tmp/wrf_management/data_folder/runs/2018_02_19/2019-02-21T17-29-01_753696_ungrib


In [10]:
ut.get_tb_from_name(tb_name='run_unique_id')

,unique_id,comments,program,run_name
0,2019-02-21T15-04-27_476148_geogrid,exploring geogrid,geogrid,2018_02_19
1,2019-02-21T17-29-01_753696_ungrib,first run,ungrib,2018_02_19


In [11]:
date_to_ungrib = '2017-12-15'

In [12]:
rows = {}
for tb in gc.FILE_TYPES.keys():
#     print(tb)
    sql = """
    select * from {tb} where date(date)=date('{dt}')
    """.format(tb=tb,dt=date_to_ungrib)
    rows[tb]=pd.read_sql(sql,con)
    rows[tb]['tb']=tb
df = pd.concat(list(rows.values()))
if df.downloaded.all()!=True: print('not all downloaded')
df

,date,i,downloaded,name,untarred,tb
0,2017-12-15 00:00:00,14,1,cdas1.20171215.pgrbh.tar,0,press
0,2017-12-15 00:00:00,14,1,cdas1.20171215.splgrbf.tar,0,surf_0
0,2017-12-15 00:00:00,14,1,cdas1.20171215.sfluxgrbf.tar,0,surf_1


In [13]:
df['tar_path']=df.apply(lambda r: os.path.join(ut.get_tar_path(r.tb),r['name']),
         axis = 1)
df['untar_path']=df.apply(lambda r: new_path,
         axis = 1)
row = df.iloc[0]
df

,date,i,downloaded,name,untarred,tb,tar_path,untar_path
0,2017-12-15 00:00:00,14,1,cdas1.20171215.pgrbh.tar,0,press,/tmp/wrf_management/data_folder/press_tar/cdas...,/tmp/wrf_management/data_folder/runs/2018_02_1...
0,2017-12-15 00:00:00,14,1,cdas1.20171215.splgrbf.tar,0,surf_0,/tmp/wrf_management/data_folder/surf_0_tar/cda...,/tmp/wrf_management/data_folder/runs/2018_02_1...
0,2017-12-15 00:00:00,14,1,cdas1.20171215.sfluxgrbf.tar,0,surf_1,/tmp/wrf_management/data_folder/surf_1_tar/cda...,/tmp/wrf_management/data_folder/runs/2018_02_1...


In [368]:
import tarfile
tfile = tarfile.TarFile(row.tar_path)

In [ ]:
tfile.extractall(row.untar_path)